In [ ]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define Project Folder
FOLDERNAME = 'Colab\ Notebooks/SC201L16'

%cd drive/MyDrive/$FOLDERNAME

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/SC201L16


In [ ]:
# Define device
import torch
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print('Device:', device)

Device: cuda


In [ ]:
# Load Existing Dataset
import torchvision.datasets as dset
import torchvision.transforms as T
transform = T.ToTensor()
train_data = dset.MNIST(root='./train', train=True, download=True, transform=transform)
val_data = dset.MNIST(root='./val', train=False, download=True, transform=transform)

In [ ]:
# Check Data Dimension
print(train_data[0][0].shape)
print(val_data[0][0].shape)

torch.Size([1, 28, 28])
torch.Size([1, 28, 28])


In [ ]:
num_train = len(train_data)
num_val = len(val_data)
print('Number of training:', num_train)
print('Number of validation:', num_val)

Number of training: 60000
Number of validation: 10000


In [ ]:
# Build Model
import torch.nn as nn

class MyRNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    # N x 1 x H x W
    x = x.squeeze()
    # N x H x W
    output, (h_n, c_n) = self.lstm(x)
    out = output[:, -1, :]
    out = self.fc(out)
    return out

model = MyRNN(28, 128, 10)
# class MyModel(nn.Module):
#   def __init__(self):
#     super().__init__()
#     #Store layers iwth weights to self
#     self.conv2d_obj = nn.Conv2d(1, 64, 3, 1, 1)
#     self.bn2d_obj = nn.BatchNorm2d(64)
#     self.fc_obj = nn.Linear(64*14*14, 10)

#   def forward(self, x):
#     out = self.conv2d_obj(x)
#     out = self.bn2d_obj(out)
#     out = nn.functional.relu(out)
#     out = nn.functional.max_pool2d(out, (2, 2))
#     out = torch.flatten(out, start_dim=1)
#     out = self.fc_obj(out)
#     return out

# model = MyModel()


# model = None
# model = nn.Sequential(
#     # N x 1 x 28 x 28
#     nn.Conv2d(1, 64, 3, 1, 1),
#     nn.BatchNorm2d(64),
#     nn.ReLU(),
#     nn.MaxPool2d(2, 2),
#     # N x 64 x 14 x 14
#     nn.Flatten(),
#     nn.Linear(64*14*14, 10)
# )

In [ ]:
# Move model to GPU
model = model.to(device)

In [ ]:
# Create Mini-batches
from torch.utils.data import DataLoader
mini_trains = DataLoader(train_data, batch_size=128, shuffle=True)
mini_vals = DataLoader(val_data, batch_size=128, shuffle=True)

In [ ]:
# Training Procedure
def train(num_epoch, model, mini_trains, mini_vals, device, loss_function, optimizer):
  for epoch in range(num_epoch):
    num_iters = 0
    for x, y in mini_trains:
      model.train()
      x = x.to(device)
      y = y.to(device)
      scores = model(x)
      loss = loss_function(scores, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if num_iters % 10 == 0:
        evaluate_predictor(model, epoch, mini_vals, device)
      num_iters += 1

In [ ]:
# Validating Procedure
def evaluate_predictor(model, epoch, mini_vals, device):
  model.eval()
  with torch.no_grad():
    acc_count = 0
    for x, y in mini_vals:
      x = x.to(device)
      y = y.to(device)
      scores=model(x)
      predictions=scores.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_count += acc
    print(f'Epoch[{epoch+1}] Acc: {acc_count/num_val}')

In [ ]:
# Define loss function & optimizer
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Start training
train(1, model, mini_trains, mini_vals, device, loss_function, optimizer)

Epoch[1] Acc: 0.0938
Epoch[1] Acc: 0.1864
Epoch[1] Acc: 0.1071
Epoch[1] Acc: 0.3161
Epoch[1] Acc: 0.4508
Epoch[1] Acc: 0.5171
Epoch[1] Acc: 0.6057
Epoch[1] Acc: 0.6447
Epoch[1] Acc: 0.7317
Epoch[1] Acc: 0.7617
Epoch[1] Acc: 0.7866
Epoch[1] Acc: 0.7836
Epoch[1] Acc: 0.8318
Epoch[1] Acc: 0.8529
Epoch[1] Acc: 0.8533
Epoch[1] Acc: 0.8598
Epoch[1] Acc: 0.8789
Epoch[1] Acc: 0.8735
Epoch[1] Acc: 0.8815
Epoch[1] Acc: 0.8879
Epoch[1] Acc: 0.8856
Epoch[1] Acc: 0.8827
Epoch[1] Acc: 0.8769
Epoch[1] Acc: 0.8874
Epoch[1] Acc: 0.9012
Epoch[1] Acc: 0.906
Epoch[1] Acc: 0.904
Epoch[1] Acc: 0.9092
Epoch[1] Acc: 0.9033
Epoch[1] Acc: 0.9167
Epoch[1] Acc: 0.9002
Epoch[1] Acc: 0.9138
Epoch[1] Acc: 0.9216
Epoch[1] Acc: 0.9048
Epoch[1] Acc: 0.9204
Epoch[1] Acc: 0.9251
Epoch[1] Acc: 0.9253
Epoch[1] Acc: 0.9278
Epoch[1] Acc: 0.9311
Epoch[1] Acc: 0.9357
Epoch[1] Acc: 0.9323
Epoch[1] Acc: 0.9259
Epoch[1] Acc: 0.9204
Epoch[1] Acc: 0.9299
Epoch[1] Acc: 0.9298
Epoch[1] Acc: 0.9338
Epoch[1] Acc: 0.9346
